In [1]:
import numpy as np 
import pandas as pd
df = pd.read_csv('./datasets/weather/weather.csv')
df

,date,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC),OT
0,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,...,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45,428.1
1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,...,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51,428.0
2,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,...,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60,427.6
3,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,...,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70,430.0
4,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,...,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81,432.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52691,2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,...,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40,433.0
52692,2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,...,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42,439.6
52693,2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,...,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45,435.2
52694,2020-12-31 23:50:00,978.26,2.07,276.95,-0.77,81.4,7.09,5.77,1.32,3.68,...,0.57,1.07,196.6,0.0,0.0,0.0,0.0,0.0,13.47,433.9


In [2]:
x, y = df.to_numpy()[:, 1:-1], df.to_numpy()[:, -1]
from sklearn.preprocessing import MinMaxScaler
x = MinMaxScaler().fit_transform(x)

In [6]:
seq_len = 96
border1s = [0, 12 * 30 * 24 * 4 - seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - seq_len]
border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]
border1s, border2s

([0, 34464, 45984], [34560, 46080, 57600])

In [ ]:
def create_window_dataset(data, target, window_size, forecast_size):
    """
    将时间序列数据转换为窗口数据集
    :param data: 特征数据（numpy数组）
    :param target: 目标数据（numpy数组）
    :param window_size: 输入窗口大小（历史时间步数）
    :param forecast_size: 预测窗口大小（未来时间步数）
    :return: 窗口化的特征数据和目标数据
    """
    X, y = [], []
    total_samples = len(data) - window_size - forecast_size + 1
    
    for i in range(total_samples):
        # 获取当前窗口的特征数据
        X_window = data[i:i+window_size]
        # 获取对应的未来窗口目标数据
        y_window = target[i+window_size : i+window_size+forecast_size]
        
        X.append(X_window)
        y.append(y_window)
    
    return np.array(X), np.array(y)

# 设置窗口参数（可根据需求调整）
WINDOW_SIZE = 5    # 使用过去5个时间步
FORECAST_SIZE = 3  # 预测未来3个时间步

# 创建窗口数据集
X_window, y_window = create_window_dataset(x, y, WINDOW_SIZE, FORECAST_SIZE)

# 查看数据集形状
print("特征数据形状:", X_window.shape)  # (样本数, 窗口大小, 特征数)
print("目标数据形状:", y_window.shape) # (样本数, 预测窗口大小)

In [ ]:
import torch
x = torch.rand(10, 1)
x = x.to(torch.device("mps"))
# Check if MPS is available
print(torch.backends.mps.is_available())

In [ ]:
torch.device("mps")

In [ ]:
torch.device("cpu")

In [ ]:
torch.device("cuda")

In [ ]:
torch.device("cuda")

In [3]:
# 导入必要的库
import os  # 用于路径操作
import pandas as pd  # 用于数据处理
from sklearn.preprocessing import StandardScaler  # 用于标准化数据

# 读取数据并进行处理
scaler = StandardScaler()  # 初始化标准化器
df_raw = pd.read_csv('./datasets/weather/weather.csv')  # 读取原始数据
set_type = 'OT'
seq_len = 96 
# 定义数据集划分的边界索引
border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
border1 = border1s[set_type]  # 根据设置类型选择边界1
border2 = border2s[set_type]  # 根据设置类型选择边界2

# 根据特征类型选择数据列
if features == 'M' or features == 'MS':  # 多变量特征（M）或多变量+目标特征（MS）
    cols_data = df_raw.columns[1:]  # 选择除第一列外的所有列
    df_data = df_raw[cols_data]  # 提取数据
elif features == 'S':  # 单变量特征（S）
    df_data = df_raw[[target]]  # 仅选择目标列

# 判断是否需要进行数据标准化
if scale:
    train_data = df_data[border1s[0]:border2s[0]]  # 使用训练集数据进行标准化
    scaler.fit(train_data.values)  # 拟合标准化器
    data = scaler.transform(df_data.values)  # 标准化数据
else:
    data = df_data.values  # 不进行标准化，直接获取数据

# 处理时间戳信息
df_stamp = df_raw[['date']][border1:border2]  # 提取时间戳数据
df_stamp['date'] = pd.to_datetime(df_stamp.date)  # 转换为datetime格式

# 根据时间编码方式提取时间特征
if timeenc == 0:  # 如果是传统时间编码
    df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)  # 提取月份
    df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)  # 提取日期
    df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)  # 提取星期几
    df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)  # 提取小时
    data_stamp = df_stamp.drop(['date'], 1).values  # 删除日期列，获取时间特征数据
elif timeenc == 1:  # 如果使用自定义时间特征提取
    data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=freq)  # 提取时间特征
    data_stamp = data_stamp.transpose(1, 0)  # 转置，使得时间特征维度正确

# 依据边界截取数据
data_x = data[border1:border2]  # 输入数据
data_y = data[border1:border2]  # 目标数据（此处默认输入和目标相同）
data_stamp = data_stamp  # 时间特征数据

TypeError: list indices must be integers or slices, not str